# バイアスの推定を使ったアルゴリズム

参考：
* [Regret Minimization for Reinforcement Learning by Evaluating the Optimal Bias Function](https://arxiv.org/abs/1906.05110)

今回は平均報酬でもバイアスを推定するアプローチです．

最適なバイアス関数があれば良いリグレットが達成できますが，バイアス関数は一般に未知です．
そこで，バイアスを推定するアプローチについて考えてみます．
今回は$\operatorname{sp}(h^*)$の最大値がわかっている場合を考えます（普通はわからない）

## バイアスの推定機

最適な行動を選択し続ける状況について考えましょう．ここで，２つの状態$s, s'$について，$s$を時刻$t_1$で出発し，$s'$に時刻$t_2$で初めて到着するとします（$t_2$が停止時刻です）．
このとき，定義から明らかに
$$
\mathbb{E}\left[\sum_{t=t_1}^{t_2-1}\left(r_t-\rho^*\right)\right]=\delta_{s, s^{\prime}}^*:=h_s^*-h_{s^{\prime}}^*
$$
が成立しています．
つまり，$\sum_{t=t_1}^{t_2-1}\left(r_t-\rho^*\right)$は$\delta^*_{s, s'}$の普遍推定です．
そこで，次の定義と補題を考えてみましょう．

---

**定義**
* 軌跡：$\mathcal{L}=\left\{\left(s_t, a_t, s_{t+1}, r_t\right)\right\}_{1 \leq t \leq N}$ 
* $s \neq s^{\prime}$を満たす$s, s^{\prime} \in \mathcal{S}$
* $\operatorname{ts}_1(\mathcal{L}):=\min \left\{\min \left\{t \mid s_t=s\right\}, N+2\right\}$：$t=1 ... N+1$の中で，$s_t=s$になった初めの時刻
* $\left\{\operatorname{ts}_k(\mathcal{L})\right\}_{k \geq 2}$：$s_t = s$, $s_{t'}=s'$を過去に$k-1$回繰り返し，そして$s_t=s$になった初めの時刻$t$
* $\left\{\operatorname{te}_k(\mathcal{L})\right\}_{k \geq 1}$：$s_t = s$, $s_{t'}=s'$を過去に$k-1$回繰り返し，そして$s_t=s$になり，続いて$s_{t'}=s'$になった初めの時刻$t'$

$$
\begin{aligned}
\operatorname{t e}_k(\mathcal{L}) & :=\min \left\{\min \left\{t \mid s_t=s^{\prime}, t>\operatorname{t s}_k(\mathcal{L})\right\}, N+2\right\}, \\
\operatorname{t s}_k(\mathcal{L}) & :=\min \left\{\min \left\{t \mid s_t=s, t>\operatorname{t e}_{k-1}(\mathcal{L})\right\}, N+2\right\}
\end{aligned}
$$

* $c\left(s, s^{\prime}, \mathcal{L}\right)$：軌跡の中で$s\to s'$を繰り返した回数
$$
c\left(s, s^{\prime}, \mathcal{L}\right):=\max \left\{k \mid \operatorname{t e}_k(\mathcal{L}) \leq N+1\right\}
$$

---

**補題**

* あるMDPで，任意の行動が最適な行動のとき，そのMDPをflatと呼ぶ．
* $M$をflatなMDPとし，そこでアルゴリズム$\mathcal{G}$を$N$回走らせるとしよう．
* $\mathcal{L}=\left\{\left(s_t, a_t, s_{t+1}, r_t\right)\right\}_{1 \leq t \leq N}$を軌跡とする．

このとき，任意の$\mathcal{G}$で，高確率で，任意の$1 \leq c \leq c\left(s, s^{\prime}, \mathcal{L}\right)$について次が成立する：

$$\left|\sum_{k=1}^c\left(\underbrace{h_{s^{\prime}}^*-h_s^*}_{真のスパン}+\underbrace{\sum_{\operatorname{ts}_k(\mathcal{L}) \leq t \leq \operatorname{te}_k(\mathcal{L})-1}\left(r_t-\rho^*\right)}_{スパンの推定}\right)\right| \leq(\sqrt{2 N \gamma}+1) \operatorname{sp}\left(h^*\right)$$

ここで$\gamma=\log \left(\frac{2}{\delta}\right)$．

証明はTODO（多分Hoeffdingを使うだけ？）




---

これを使って$\mathcal{H}_k$を計算することを考えましょう．このとき，
1. $M$がflatではないかも
2. $\rho^*$を知らない
   
の２つの問題があります．これを解決しましょう．

まず，ベルマン方程式から，
$$
h_s^*=\max_{a}\left\{r_{s, a} + P_{s, a}^T h^*\right\}-\rho^*
$$
が成り立ちます．つまり，
$$\operatorname{reg}_{s, a}
=h_s^*+\rho^*-P_{s, a}^T h^*-r_{s, a}
=\max_{a}\left\{r_{s, a} + P_{s, a}^T h^*\right\}-P_{s, a}^T h^*-r_{s, a}
$$
は$(s, a)$を選択した場合の最適性からの差を表します．
$$r'_{s, a} = r_{s, a} + \operatorname{reg}_{s, a} = h_s^*+\rho^*-P_{s, a}^T h^*$$
としましょう．

このとき，MDP $M'=\langle P, r'\rangle$はflatであり，最適バイアスと最適ゲインが$M$と同じであることが言えます．

**↑の証明**

$M'$のベルマン方程式を変形すると，

$$
\begin{aligned}
h_s^{\prime *}
&=\max_{a}\left\{r'_{s, a} + P_{s, a}^T h^{\prime *}\right\}-\rho^{\prime *}\\
&=\max_{a}\left\{ h_s^*+\rho^*-P_{s, a}^T h^* + P_{s, a}^T h^{\prime *}\right\}-\rho^{\prime *}\\
&=h_s^*+ \max_{a}\left\{-P_{s, a}^T h^* + P_{s, a}^T h^{\prime *}\right\}+\rho^* -\rho^{\prime *}\\
\end{aligned}
$$
を満たします．
これは明らかに$h^{\prime *} = h^*$かつ$\rho^{\prime *}=\rho^*$がベルマン方程式の解なので，$M$と同じです．

また，その場合はどの行動を選択しても$\max_a$の中身がゼロになるので，どの行動でもベルマン方程式が満足されます．よって，どの行動も最適であり，このMDPはflatです．

---


よって，上でやった補題から，

$$
\left|\sum_{k=1}^c\left(h_{s^{\prime}}^*-h_s^*+\sum_{\operatorname{ts}_k(\mathcal{L}) \leq t \leq \operatorname{te}_k(\mathcal{L})-1}\left(r_t-\rho^*\right)\right)\right| \leq(\sqrt{2 N \gamma}+1) \operatorname{sp}\left(h^*\right) + \sum_{t=1}^N \operatorname{reg}_{s_t, a_t}$$

が成立します．ここで，$h'\in [0, H]^S$を，$h^*$から$h'$に入れ替えても上の不等式が成り立つようなバイアス関数とすると，結局
$$
N_{s, a, s'}\left|\left(h_{s^{\prime}}^*-h_s^*\right)+\left(h_{s^{\prime}}^\prime-h_s^\prime\right) \right| \leq 2(\sqrt{2 N \gamma}+1) H + 2 \sum_{t=1}^N \operatorname{reg}_{s_t, a_t}$$

が言えます．ここで，
$$
N_{s, a, s^{\prime}}:=\sum_{t=1}^N \mathbb{I}\left[s_t=s, a_t=a, s_{t+1}=s^{\prime}\right] \leq c\left(s, s^{\prime}, \mathcal{L}\right)
$$
としました．この不等式のアルゴリズムは何でも良いので，REGAL.Cアルゴリズムで動いたとすると，$\sum_{t=1}^N \operatorname{reg}_{s_t, a_t} \leq \tilde{O}(H S \sqrt{A N})$とできるので，高確率で

$$
\hat{N}_{s, a, s^{\prime}}\left|\left(h_{s^{\prime}}^*-h_s^*\right)-\left(h_{s^{\prime}}^{\prime}-h_s^{\prime}\right)\right|=\tilde{O}(H S \sqrt{A N})
$$

が言えます．



---

以上を踏まえて，
$$
\mathcal{H}_k:=\left\{h \in[0, H]^S| | L_1\left(h, s, s^{\prime}, \mathcal{L}_{t_k-1}\right) \mid \leq 48 S \sqrt{A T} s p(h)+(\sqrt{2 \gamma T}+1) \operatorname{sp}(h), \forall s, s^{\prime}, s \neq s^{\prime}\right\}
$$
によってバイアスの信頼区間を作ることを考えます．ここで，
$$
L_1\left(h, s, s^{\prime}, \mathcal{L}\right)=\sum_{k=1}^{c\left(s, s^{\prime}, \mathcal{L}\right)}\left(\left(h_{s^{\prime}}-h_s\right)+\sum_{t s_k(\mathcal{L}) \leq i \leq t e_k(\mathcal{L})-1}\left(r_i-\hat{\rho}\right)\right)
$$
としました．

まとめると，次の形式で遷移の信頼区間を作ります：
任意の$s, a, s', h' \in \mathcal{H}$について，

* Bernstein：$\left|P_{s, a, s^{\prime}}^{\prime}(\pi)-\hat{P}_{s, a, s^{\prime}}\right| \leq 2 \sqrt{\hat{P}_{s, a, s^{\prime}} \gamma / N_{s, a}}+3 \gamma / N_{s, a}+4 \gamma^{\frac{3}{4}} / N_{s, a}^{\frac{3}{4}}$
* $L_1$ノルム：$\left|P_{s, a}^{\prime}(\pi)-\hat{P}_{s, a}\right|_1 \leq \sqrt{14 S \gamma / N_{s, a}}$
* スパン：$\left|\left(P_{s, a}^{\prime}(\pi)-\hat{P}_{s, a}\right)^T h^{\prime}(\pi)\right| \leq 2 \sqrt{V\left(\hat{P}_{s, a}, h^{\prime}(\pi)\right) \gamma / N_{s, a}}+12 H \gamma / N_{s, a}+10 H \gamma^{3 / 4} / N_{k, s, a}^{3 / 4}$
* Extended Value iteration：$P_{s, \pi(s)}^{\prime}(\pi)^T h^{\prime}(\pi)+r_{s, \pi(s)}=\max _{a \in \mathcal{A}} P_{s, a}^{\prime}(\pi)^T h^{\prime}(\pi)+r_{s, a}=h^{\prime}(\pi)+\rho(\pi) \mathbf{1}$

**コメント：これはどうやって実装すればいいかよくわからんな… 実際，論文の中で効率的な実装はよくわからないのでFuture Workって言ってる**

## リグレット解析

最終的に次のリグレットを出します：
$$
\mathcal{R}(T) \leq 490 \sqrt{S A H T \log \left(\frac{40 S^2 A^2 T \log (T)}{\delta}\right)}
$$

まず，次のBad eventが低確率で生じることを言いましょう：

$$
\begin{aligned}
& B_{1, k}:=\left\{\exists(s, a), \text { s.t. }\left|\left(P_{s, a}-\hat{P}_{s, a}^{(k)}\right)^T h^*\right|>2 \sqrt{\frac{\left.V\left(P_{s, a}, h^*\right) \gamma\right)}{\max \left\{N_{k, s, a}, 1\right\}}}+2 \frac{s p\left(h^* \gamma\right)}{\max \left\{N_{k, s, a}, 1\right\}}\right\}, \\
& B_{2, k}=\left\{\exists\left(s, a, s^{\prime}\right), \text { s.t. }\left|\hat{P}_{s, a, s^{\prime}}^{(k)}-P_{s, a, s^{\prime}}\right|>2 \sqrt{\frac{\hat{P}_{s, a, s^{\prime}}^{(k)} \gamma}{\max \left\{N_{k, s, a}, 1\right\}}}+\frac{3 \gamma}{\max \left\{N_{k, s, a}, 1\right\}}+\frac{4 \gamma^{\frac{3}{4}}}{\max \left\{N_{k, s, a}, 1\right\}^{\frac{3}{4}}}\right\}, \\
& B_{3, k}=\left\{\left|\sum_{1 \leq t<t_k}\left(\rho^*-r_{s_t, a_t}\right)\right|>26 H S \sqrt{A T \gamma}, \sum_{k^{\prime}<k} \sum_{s, a} v_{k^{\prime}, s, a} r e g_{s, a}>22 H S \sqrt{A T \gamma}\right\} \\
& B_{4, k}=\left\{\left\{\left(\pi^*, P^*, h^*, \rho^*\right) \mid \pi^* \text { is a deterministic optimal policy }\right\} \cap \mathcal{M}_k=\varnothing\right\} .
\end{aligned}
$$

ここの細かいBad eventの確率バウンドの証明は任せます．（多分そんなにむずくない）

$B_{4, k}$を使ってRegretを次のように分解します（$v_k$はvisitationのcountベクトル）：

$$
\begin{aligned}
\mathcal{R}_k & =v_k^T\left(\rho^* \mathbf{1}-r_k\right) \leq v_k^T\left(\rho_k \mathbf{1}-r_k\right)=v_k^T\left(P_k^{\prime}-I\right)^T h_k \\
& =\underbrace{v_k^T\left(P_k-I\right)^T h_k}_{(1)_k}+\underbrace{v_k^T\left(\hat{P}_k-P_k\right)^T h^*}_{(2)_k}+\underbrace{v_k^T\left(P_k^{\prime}-\hat{P}_k\right)^T h_k}_{(3)_k}+\underbrace{v_k^T\left(\hat{P}_k-P_k\right)^T\left(h_k-h^*\right)}_{(4)_k}
\end{aligned}
$$

---

**$(1)_k$のバウンド**

$$
\begin{aligned}
(1)_k&=v_k^T\left(P_k-I\right)^T h_k
=\sum_{i=1}^n\left(P_{s_i, a_i}^T h_k-h_{k, s_i}\right)\\
&=\sum_{i=1}^{l_k}\left(P_{s_i, a_i}^T h_k-h_{k, s_{i+1}}\right)-
h_{k, s_1}+h_{k, s_{I_k+1}}
\end{aligned}
$$

一項目はMartingaleで抑えられそう．二項目は単純に$HK$ばすれば良し．
$K$はdoubling trickで抑えられるので大丈夫：
$$
K \leq S A\left(\log _2\left(\frac{T}{S A}\right)+1\right) \leq 3 S A \log (T)
$$

**$(2)_k$のバウンド**
Bernsteinで簡単
$$
\left(2_k \leq \sum_{s, a} v_{k, s, a}\left(2 \sqrt{\frac{V\left(P_{s, a}, h^*\right) \gamma}{\max \left\{N_{k, s, a}, 1\right\}}}+2 \frac{H \gamma}{\max \left\{N_{k, s, a}, 1\right\}}\right) \approx O\left(\sum_{s, a} v_{k, s, a} \sqrt{\frac{V\left(P_{s, a}, h^*\right) \gamma}{\max \left\{N_{k, s, a}, 1\right\}}}\right)\right.
$$

**$(3)_k$のバウンド**

スパンについての信頼区間の作り方を踏まえると，

$$
\text { (3) } k \leq \sum_{s, a} v_{k, s, a} L_2\left(\max \left\{N_{k, s, a}, 1\right\}, \hat{P}_{s, a}^{(k)}, h_k\right) \approx O\left(\sum_{s, a} v_{k, s, a} \sqrt{\frac{V\left(\hat{P}_{s, a}^{(k)}, h_k\right) \gamma}{\max \left\{N_{k, s, a}, 1\right\}}}\right)
$$

が成立します．後は算数．


**$(4)_k$のバウンド**

$$
\begin{aligned}
(4)_k & =\sum_{s, a} v_{k, s, a}\left(\hat{P}_{s, a}^{(k)}-P_{s, a}\right)^T\left(h_k-h_{k, s} \mathbf{1}-h^*+h_s^* \mathbf{1}\right)=\sum_{s, a} v_{k, s, a} \sum_{s^{\prime}}\left(\hat{P}_{s, a, s^{\prime}}^{(k)}-P_{s, a, s}\right)\left(\delta_{s, s^{\prime}}^*-\delta_{k, s, s^{\prime}}\right) \\
& \approx O\left(\sum_{s, a} v_{k, s, a} \sum_{s^{\prime}} \sqrt{\left.\frac{\hat{P}_{s, a, s^{\prime}}^{(k)} \gamma}{\max \left\{N_{k, s, a}, 1\right\}}\left|\delta_{k, s, s^{\prime}}-\delta_{s, s^{\prime}}^*\right|\right)}\right. \\
& =O\left(\sqrt{H} \sum_{s, a} v_{k, s, a} \sum_{s^{\prime}} \sqrt{\frac{\hat{P}_{s, a, s^{\prime}}^{(k)} \gamma\left|\delta_{k, s, s^{\prime}}-\delta_{s, s^{\prime}}^*\right|}{\max \left\{N_{k, s, a}, 1\right\}}}\right) .
\end{aligned}
$$